# Example: Ingesting Digital Assets Data with a Third-Party Integration

[CryptoFeed](https://github.com/bmoscon/cryptofeed) is an open-source library for retrieving data from multiple digital asset exchange feeds and storing it in a high-performance database. One of its supported backends is QuestDB.

In this example, we’ll capture trade data for the symbols `ADA-USDT`, `BTC-USDT`, `ETH-BTC`, and `ETH-USDT` from [OKX](https://www.okx.com/), and store it in QuestDB.

Since CryptoFeed handles the entire ingestion process, you only need to point it to the QuestDB TCP socket (which defaults to port 9009). The library will take care of the rest.

When you start the notebook, it will begin capturing data continuously until you either stop the execution or close the browser tab.

This dataset is the same one powering the public demo at `https://demo.questdb.io`, with the only difference being that the demo machine tracks 32 symbols instead of just four.

The script below processes each message and stores it in a table named `trades` in QuestDB. If the table doesn’t exist, it will be automatically created on the first write:

```sql
CREATE TABLE 'trades' (
	symbol SYMBOL CAPACITY 256 CACHE,
	side SYMBOL CAPACITY 256 CACHE,
	price DOUBLE,
	amount DOUBLE,
	timestamp TIMESTAMP
) timestamp(timestamp) PARTITION BY DAY WAL;
```

To view the live data in your database, open a new browser tab and navigate to `http://localhost:9000`. You can run a basic query like:

```sql
SELECT * FROM trades -10;
```

to see the latest 10 trades. For something more insightful, try:

```sql
SELECT timestamp, symbol, side, sum(price * amount)
FROM trades
SAMPLE BY 1m;
```

This gives you trade volume totals per symbol at 1-minute intervals.

For more realistic queries, open the [Examples-of-market-data-queries notebook](/notebooks/Examples-of-market-data-queries.ipynb) in a new tab. It includes adapted queries from the demo environment that should return meaningful results on your local dataset.

To visualize your live data in real time, we offer a sample dashboard powered by Grafana and another one powered by Pulse. 

To see the Grafana dashboard navigate to the [demo Grafana dashboard](http://localhost:3000/d/live-trades-demo/live-trades-demo). The user is `admin` and the password is `quest`.

To see your live data on a Pulse dashboard, please navigate in a new tab to [the demo Pulse dashboard](http://localhost:8080/dash/29/Live%20Trades%20Demo).


In [4]:
#
# This product based on cryptofeed https://github.com/bmoscon/cryptofeed software developed by Bryant Moscon (http://www.bryantmoscon.com/) Copyright (C) 2017-2022 Bryant Moscon - bmoscon@gmail.com
#

import multiprocessing
import os
import argparse
from datetime import datetime


import ipywidgets as widgets
from IPython.display import display
import threading
import time
from cryptofeed import FeedHandler
from cryptofeed.backends.backend import BackendCallback
from cryptofeed.backends.socket import SocketCallback
from cryptofeed.defines import TRADES
from cryptofeed.exchanges import OKX

QUEST_HOST = 'questdb'
QUEST_PORT = 9009

class QuestCallback(SocketCallback):
    def __init__(self, host='host.docker.internal', port=9009, **kwargs):
        super().__init__(f"tcp://{host}", port=port, **kwargs)
        self.numeric_type = float
        self.none_to = None

    async def writer(self):
        while True:
            try:
                await self.connect()
            except Exception as e:
                print("Connection error:", e)
                break
            async with self.read_queue() as update:
                update = "\n".join(update) + "\n"
                try:
                    self.conn.write(update.encode())
                except Exception as e:
                    print("Write error:", e)
                    break  

class TradeQuest(QuestCallback, BackendCallback):
    default_key = 'trades'

    async def write(self, data):
        update = f'{self.key},symbol={data["symbol"]},side={data["side"]} price={data["price"]},amount={data["amount"]} {int(data["timestamp"] * 1_000_000_000)}'
        await self.queue.put(update)
        
def run_feed():
    # In a separate process, we don't need to worry about Jupyter's event loop.
    handler = FeedHandler()
    symbols = [
        'ADA-USDT',
        'BTC-USDT',       
        'ETH-BTC',
        'ETH-USDT',
    ]
    
    handler.add_feed(OKX(channels=[TRADES], symbols=symbols,
                           callbacks={TRADES: TradeQuest(host=QUEST_HOST, port=QUEST_PORT)}))
    handler.run()


# Start the feed handler in a separate process.
feed_process = multiprocessing.Process(target=run_feed)
feed_process.start()

print("The process will run in the background until the notebook is stopped. Status will update every second")

# Create a status indicator widget
status = widgets.Label("Feed is running...")
display(status)

# Start a background thread that periodically updates the status label
def update_status():
    while feed_process.is_alive():
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        status.value = f"Feed is running... Last check: {timestamp}"
        time.sleep(1)
    status.value = "Feed stopped."


threading.Thread(target=update_status, daemon=True).start()


The process will run in the background until the notebook is stopped. Status will update every second


Label(value='Feed is running...')